In [1]:
# import splitfolders
# import os
# path = '/Users/namanh.ngco/Documents/FPT/AIL/emotion_detection/FER13_cleaner'
# print(os.listdir(path))

In [2]:
# splitfolders.ratio(path, output='data', ratio=(0.8, 0.1, 0.1))

In [3]:
# display, transform, read, split ...
import numpy as np
import cv2 as cv
import os
import splitfolders
import matplotlib.pyplot as plt

# tensorflow
import tensorflow.keras as keras
import tensorflow as tf

# image processing
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

# model / neural network
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import regularizers

In [4]:
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import concatenate
from keras.optimizers import Adam, SGD
from keras.regularizers import l1, l2
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix


In [5]:
path = './output/'

In [18]:
data_path = './output/train/'
classes = os.listdir(data_path) #['happy', 'sad', 'fear', 'surprise', 'neutral', 'angry', 'disgust']
classes.remove('.DS_Store')
# print(classes)
# len_of_class = []
# for folder in classes:
#     if folder == '.DS_Store':
#         continue
#     len_of_class.append(len(os.listdir(data_path+folder)))
# print(len_of_class)

In [7]:
datagen = ImageDataGenerator(rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

In [8]:
# Define data augmentation function
def data_augmentation(img_path: str, step: int, save_to_dir, save_format):
    # load img to array
    img = load_img(img_path)
    img_array = img_to_array(img)
    img_array = img_array.reshape((1,) + img_array.shape)

    # augmentation
    i = 0
    for batch in datagen.flow(img_array, batch_size=1, save_to_dir=save_to_dir, save_format=save_format):
        i += 1
        if i > step:
            break

In [9]:
def img_path_list(path: str):
    img_list = os.listdir(path)
    img_path_list = []
    for img in img_list:
        img_path_list.append(path + img)
    return img_path_list

In [10]:
# disgust_img_train_list = img_path_list(data_path + 'train/' + 'disgust/')
# print(disgust_img_train_list)

In [11]:
# disgust_img_val_list = img_path_list(data_path + 'val/' + 'disgust/')
# print(disgust_img_val_list)

In [12]:
# for file in disgust_img_train_list:
#     data_augmentation(file, 2, data_path + 'train/' + 'disgust/', 'jpg')
# print(len(os.listdir(data_path + 'train/' + 'disgust')))

In [13]:
# for file in disgust_img_val_list:
#     data_augmentation(file, 2, data_path + 'val/' + 'disgust/', 'jpg')
# print(len(os.listdir(data_path + 'val/' + 'disgust/')))

In [19]:
train_generator = datagen.flow_from_directory(
    directory = path + 'train/',
    classes = classes,
    batch_size = 32,
    target_size = (48, 48),
    class_mode = 'binary',
    color_mode = "grayscale",
)

Found 12167 images belonging to 4 classes.


In [20]:
test_generator = datagen.flow_from_directory(
    directory = path + 'test/',
    classes = classes,
    batch_size = 32,
    target_size = (48, 48),
    class_mode = 'binary',
    color_mode = "grayscale",

)

Found 1649 images belonging to 4 classes.


In [21]:
valid_generator = datagen.flow_from_directory(
    directory = path + 'val/',
    classes = classes,
    batch_size = 32,
    target_size = (48, 48),
    class_mode = 'binary',
    color_mode = "grayscale",

)

Found 1652 images belonging to 4 classes.


In [22]:
# img_check = image.load_img('/Users/namanh.ngco/Documents/FPT/AIL/emotion_detection/data/train/disgust/Training_81074213.jpg', '/Users/namanh.ngco/Documents/FPT/AIL/emotion_detection/data/train/disgust/Training_94620136.jpg')
# x = img_to_array(img_check)
# x.shape

In [23]:
# resnet_50 = ResNet50(include_top=False, weights='imagenet', input_shape=(48, 48, 3))
# for layer in resnet_50.layers:
#     layer.trainable = False

In [24]:
# # build the entire model
# x = resnet_50.output
# x = layers.GlobalAvgPool2D()(x)
# x = layers.Dense(512, activation = 'relu',kernel_regularizer=regularizers.l2(0.01))(x)
# x = layers.Dropout(0.5)(x)
# x = layers.Dense(256, activation = 'relu',kernel_regularizer=regularizers.l2(0.01))(x)
# x = layers.Dropout(0.5)(x)
# x = layers.Dense(128, activation = 'relu')(x)
# x = layers.Dropout(0.5)(x)
# x = layers.Dense(64, activation = 'relu')(x)
# x = layers.Dropout(0.5)(x)
# predictions = layers.Dense(3, activation='softmax')(x)
# model = Model(inputs = resnet_50.input, outputs = predictions)

In [25]:
# def FER_Model(input_shape=(48,48,1)):
#     # first input model
#     visible = Input(shape=input_shape, name='input')
#     num_classes = 3
#     #the 1-st block
#     conv1_1 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name = 'conv1_1')(visible)
#     conv1_1 = BatchNormalization()(conv1_1)
#     conv1_2 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name = 'conv1_2')(conv1_1)
#     conv1_2 = BatchNormalization()(conv1_2)
#     pool1_1 = MaxPooling2D(pool_size=(2,2), name = 'pool1_1')(conv1_2)
#     drop1_1 = Dropout(0.3, name = 'drop1_1')(pool1_1)#the 2-nd block
#     conv2_1 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_1')(drop1_1)
#     conv2_1 = BatchNormalization()(conv2_1)
#     conv2_2 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_2')(conv2_1)
#     conv2_2 = BatchNormalization()(conv2_2)
#     conv2_3 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_3')(conv2_2)
#     conv2_2 = BatchNormalization()(conv2_3)
#     pool2_1 = MaxPooling2D(pool_size=(2,2), name = 'pool2_1')(conv2_3)
#     drop2_1 = Dropout(0.3, name = 'drop2_1')(pool2_1)#the 3-rd block
#     conv3_1 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_1')(drop2_1)
#     conv3_1 = BatchNormalization()(conv3_1)
#     conv3_2 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_2')(conv3_1)
#     conv3_2 = BatchNormalization()(conv3_2)
#     conv3_3 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_3')(conv3_2)
#     conv3_3 = BatchNormalization()(conv3_3)
#     conv3_4 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_4')(conv3_3)
#     conv3_4 = BatchNormalization()(conv3_4)
#     pool3_1 = MaxPooling2D(pool_size=(2,2), name = 'pool3_1')(conv3_4)
#     drop3_1 = Dropout(0.3, name = 'drop3_1')(pool3_1)#the 4-th block
#     conv4_1 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_1')(drop3_1)
#     conv4_1 = BatchNormalization()(conv4_1)
#     conv4_2 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_2')(conv4_1)
#     conv4_2 = BatchNormalization()(conv4_2)
#     conv4_3 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_3')(conv4_2)
#     conv4_3 = BatchNormalization()(conv4_3)
#     conv4_4 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_4')(conv4_3)
#     conv4_4 = BatchNormalization()(conv4_4)
#     pool4_1 = MaxPooling2D(pool_size=(2,2), name = 'pool4_1')(conv4_4)
#     drop4_1 = Dropout(0.3, name = 'drop4_1')(pool4_1)
    
#     #the 5-th block
#     conv5_1 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_1')(drop4_1)
#     conv5_1 = BatchNormalization()(conv5_1)
#     conv5_2 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_2')(conv5_1)
#     conv5_2 = BatchNormalization()(conv5_2)
#     conv5_3 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_3')(conv5_2)
#     conv5_3 = BatchNormalization()(conv5_3)
#     conv5_4 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_4')(conv5_3)
#     conv5_3 = BatchNormalization()(conv5_3)
#     pool5_1 = MaxPooling2D(pool_size=(2,2), name = 'pool5_1')(conv5_4)
#     drop5_1 = Dropout(0.3, name = 'drop5_1')(pool5_1)#Flatten and output
#     flatten = Flatten(name = 'flatten')(drop5_1)
#     ouput = Dense(num_classes, activation='softmax', name = 'output')(flatten)# create model 
#     model = Model(inputs =visible, outputs = ouput)
#     # summary layers
#     print(model.summary())
    
#     return model

In [26]:
visible = Input(shape=(48,48,1), name='input')
num_classes = 3
#the 1-st block
conv1_1 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name = 'conv1_1')(visible)
conv1_1 = BatchNormalization()(conv1_1)
conv1_2 = Conv2D(64, kernel_size=3, activation='relu', padding='same', name = 'conv1_2')(conv1_1)
conv1_2 = BatchNormalization()(conv1_2)
pool1_1 = MaxPooling2D(pool_size=(2,2), name = 'pool1_1')(conv1_2)
drop1_1 = Dropout(0.3, name = 'drop1_1')(pool1_1)#the 2-nd block
conv2_1 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_1')(drop1_1)
conv2_1 = BatchNormalization()(conv2_1)
conv2_2 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_2')(conv2_1)
conv2_2 = BatchNormalization()(conv2_2)
conv2_3 = Conv2D(128, kernel_size=3, activation='relu', padding='same', name = 'conv2_3')(conv2_2)
conv2_2 = BatchNormalization()(conv2_3)
pool2_1 = MaxPooling2D(pool_size=(2,2), name = 'pool2_1')(conv2_3)
drop2_1 = Dropout(0.3, name = 'drop2_1')(pool2_1)#the 3-rd block
conv3_1 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_1')(drop2_1)
conv3_1 = BatchNormalization()(conv3_1)
conv3_2 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_2')(conv3_1)
conv3_2 = BatchNormalization()(conv3_2)
conv3_3 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_3')(conv3_2)
conv3_3 = BatchNormalization()(conv3_3)
conv3_4 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv3_4')(conv3_3)
conv3_4 = BatchNormalization()(conv3_4)
pool3_1 = MaxPooling2D(pool_size=(2,2), name = 'pool3_1')(conv3_4)
drop3_1 = Dropout(0.3, name = 'drop3_1')(pool3_1)#the 4-th block
conv4_1 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_1')(drop3_1)
conv4_1 = BatchNormalization()(conv4_1)
conv4_2 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_2')(conv4_1)
conv4_2 = BatchNormalization()(conv4_2)
conv4_3 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_3')(conv4_2)
conv4_3 = BatchNormalization()(conv4_3)
conv4_4 = Conv2D(256, kernel_size=3, activation='relu', padding='same', name = 'conv4_4')(conv4_3)
conv4_4 = BatchNormalization()(conv4_4)
pool4_1 = MaxPooling2D(pool_size=(2,2), name = 'pool4_1')(conv4_4)
drop4_1 = Dropout(0.3, name = 'drop4_1')(pool4_1)

#the 5-th block
conv5_1 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_1')(drop4_1)
conv5_1 = BatchNormalization()(conv5_1)
conv5_2 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_2')(conv5_1)
conv5_2 = BatchNormalization()(conv5_2)
conv5_3 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_3')(conv5_2)
conv5_3 = BatchNormalization()(conv5_3)
conv5_4 = Conv2D(512, kernel_size=3, activation='relu', padding='same', name = 'conv5_4')(conv5_3)
conv5_3 = BatchNormalization()(conv5_3)
pool5_1 = MaxPooling2D(pool_size=(2,2), name = 'pool5_1')(conv5_4)
drop5_1 = Dropout(0.3, name = 'drop5_1')(pool5_1)#Flatten and output
flatten = Flatten(name = 'flatten')(drop5_1)
ouput = Dense(num_classes, activation='softmax', name = 'output')(flatten)# create model 
model = Model(inputs =visible, outputs = ouput)

2024-03-22 08:40:19.961324: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-03-22 08:40:19.961358: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-03-22 08:40:19.961369: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-03-22 08:40:19.961387: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-22 08:40:19.961401: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [27]:
def trainModel(model, epochs, optimizer):
    batch_size = 32
    model.compile(optimizer=optimizer, loss = "sparse_categorical_crossentropy", metrics=["accuracy"])
    return model.fit(train_generator, validation_data=valid_generator, epochs = epochs, batch_size=batch_size)

In [28]:
import tensorflow as tf

# Allow GPU memory growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

Physical devices cannot be modified after being initialized


In [29]:
with tf.device("GPU:0"):
    model_running = trainModel(model=model, epochs=10, optimizer='Adam')

Epoch 1/10


2024-03-22 08:40:24.373942: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


381/381 ━━━━━━━━━━━━━━━━━━━━ 72s 159ms/step - accuracy: 0.2939 - loss: 43.3882 - val_accuracy: 0.3771 - val_loss: 1471.6359
Epoch 2/10
248/381 ━━━━━━━━━━━━━━━━━━━━ 21s 164ms/step - accuracy: 0.2932 - loss: 394.8471

In [ ]:
del model

2024-03-22 09:53:36.364011: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-03-22 09:53:36.364037: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-03-22 09:53:36.364041: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-03-22 09:53:36.364060: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-22 09:53:36.364071: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adamw', beca

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 48, 48, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 48, 48, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 48, 48, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 24, 24, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 24, 24, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 12, 12, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 12, 12, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 12, 12, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 6, 6, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 3, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 1, 1, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 7)              │           231 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,790,921 (56.42 MB)

 Trainable params: 14,790,919 (56.42 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)